In [2]:
import pandas as pd
import numpy as np
import wfdb
import ast

In [3]:
%run target_dict.ipynb

In [4]:
# путь к файлу и кол-во семплов
path = 'ptb-xl/1.0.1/'
sampling_rate=100

In [5]:
#для формирования X по ЭКГ (кол-во ЭКГ(не пациентов!), 100 записей в сек, кол-во электродов)
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    
    return data

In [6]:
# для data_X
def load_raw_data_X(df, sampling_rate, path):
    data_X = []
    if sampling_rate == 100:
        for f in df.filename_lr:
            data_X.append([df.filename_lr, wfdb.rdsamp(path+f)])
    else:
        for f in df.filename_hr:
            data_X.append([df.filename_hr, wfdb.rdsamp(path+f)])        
    
    return data_X

In [7]:
# проба
path = 'ptb-xl/1.0.1/'
f = 'records100/00000/00001_lr'
d = wfdb.rdsamp(path+f)
d[0]

array([[-0.119, -0.055,  0.064, ..., -0.026, -0.039, -0.079],
       [-0.116, -0.051,  0.065, ..., -0.031, -0.034, -0.074],
       [-0.12 , -0.044,  0.076, ..., -0.028, -0.029, -0.069],
       ...,
       [ 0.069,  0.   , -0.069, ...,  0.024, -0.041, -0.058],
       [ 0.086,  0.004, -0.081, ...,  0.242, -0.046, -0.098],
       [ 0.022, -0.031, -0.054, ...,  0.143, -0.035, -0.12 ]])

In [8]:
# загрузка даты с лейблами
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

In [9]:
# для обозначения суперкласса нормальный/не нормальный (в задаче другие таргеты)
def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        
        if key in agg_df.index:
            diag = agg_df.loc[key].diagnostic_class
            if diag == 'NORM':
                tmp.append(diag)
           
            else:
                tmp.append('pathology')
                
    return list(set(tmp))

In [10]:
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [11]:
# нам нужны свои таргеты согласно заданию - посмотрим, где их искать и обозначения
for i in Y.scp_codes:
    print(i)

{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SBRAD': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SARRH': 0.0}
{'NORM': 80.0, 'SBRAD': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'AFLT': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'AFLT': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'AFLT': 100.0, 'ABQRS': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'AFLT': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NST_': 100.0, 'DIG': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LPFB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'AFLT': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 

{'ISCIL': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'ALMI': 100.0}
{'IMI': 80.0, 'ASMI': 80.0, 'SR': 0.0}
{'IRBBB': 100.0, 'SR': 0.0}
{'ISCAL': 100.0, 'LVH': 50.0, 'SR': 0.0}
{'ASMI': 50.0, 'CRBBB': 100.0, 'LAFB': 100.0, 'ISCAL': 50.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'LVH': 100.0, 'ISC_': 100.0, 'PVC': 100.0, 'AFIB': 0.0}
{'ASMI': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 50.0, 'LVH': 100.0, 'ISC_': 100.0, 'LAFB': 100.0, 'PVC': 100.0}
{'LAFB': 100.0, 'IRBBB': 100.0, 'SR': 0.0}
{'ISCIL': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 15.0, 'LAO/LAE': 50.0}
{'NORM': 80.0, 'SR': 0.0}
{'ISCIL': 100.0, 'SR': 0.0}
{'IRBBB': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
{'LVH': 100.0, 'SR': 0.0}
{'ASMI': 50.0, 'CRBBB': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'ISC_': 100.0}
{'IMI': 100.0, 'NDT': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'NORM': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 

{'NORM': 100.0, 'SARRH': 0.0}
{'CRBBB': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'PVC': 100.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SBRAD': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'IVCD': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'STACH': 0.0}
{'NORM': 100.0, 'IRBBB': 100.0, 'SARRH': 0.0}
{'ALMI': 100.0, 'ASMI': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SBRAD': 0.0}
{'ALMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'AMI': 80.0, 'LVH': 100.0, 'DIG': 100.0, 'VCLVH': 0.0, 'QWAVE': 0.0, 'LOWT': 0.0, 'SR': 0.0}
{'NST_': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'STACH': 0.0}
{'ASMI': 50.0, 'LMI': 15.0, 'VCLVH': 0.0, 'AFIB': 0.0}
{'LVH': 15.0, 'SR': 0.0}
{'NORM': 100.0, 'PVC': 0.0, 'STACH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 15.0, 'IMI': 15.0, 'LAFB': 100.0, 'NST_': 0.0, 'SR': 0.0}
{'ASMI': 100.0, 'ALMI': 100.0, 'IVCD': 100.0,

{'ASMI': 15.0, 'CRBBB': 100.0, 'AFIB': 0.0}
{'LVH': 35.0, 'PAC': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'LOWT': 0.0, 'AFIB': 0.0}
{'IMI': 80.0, 'QWAVE': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'STACH': 0.0}
{'NDT': 100.0, 'PAC': 0.0, 'NST_': 0.0, 'STACH': 0.0}
{'LVH': 50.0, 'NDT': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'NST_': 100.0, 'IVCD': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'LVH': 50.0, 'VCLVH': 0.0, 'SR': 0.0}
{'IMI': 80.0, 'QWAVE': 0.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IMI': 15.0, 'IRBBB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 15.0, 'INJAS': 100.0, 'INVT': 0.0, 'LOWT': 0.0, 'SR': 0.0}
{'ASMI': 15.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'LAO/LAE': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LAFB': 100.0, 'PVC': 100.0, 'LOWT': 0.0, 'STD_': 0.0, 'SR': 0.0}
{'LMI': 50.0, 'SR': 0.0}
{'ASMI': 50.0, 'LVH': 50.0, 'ISC_': 100.0, 'PVC': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'CRBBB': 100

{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'PVC': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ILMI': 100.0, 'IVCD': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 15.0, 'QWAVE': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'1AVB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'AFIB': 0.0}
{'IMI': 15.0, 'PVC': 100.0, 'SARRH': 0.0}
{'LAFB': 100.0}
{'IMI': 100.0, 'NDT': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'VCLVH': 0.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SARRH': 0.0}
{'IMI': 50.0, 'ASMI': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'IMI': 100.0, 'IVCD': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'STD_': 0.0, 'SARRH': 0.0}
{'IRBBB': 100.0, 'AFIB': 0.0}
{'INJAS': 100.0, 'INJAL': 100.0, 'LVH': 100.0, 'INVT': 0.0, 'STD_': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'NT_': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'NT_': 0.0, 'SR': 0.0}
{'IMI': 35.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'STD_': 0.0, '

...

{'IMI': 100.0, 'PVC': 100.0, 'ABQRS': 0.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'ISC_': 100.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'STACH': 0.0}
{'NST_': 100.0, 'AFIB': 0.0, 'AFLT': 0.0}
{'LAFB': 100.0, 'LVH': 100.0, 'PVC': 100.0, 'AFIB': 0.0}
{'ASMI': 100.0, 'LMI': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'IMI': 100.0, 'LVH': 100.0, 'ISC_': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'LVH': 100.0, 'STD_': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'LVH': 100.0, 'ISC_': 100.0}
{'ASMI': 15.0, 'LAFB': 100.0, 'LAO/LAE': 50.0, 'SR': 0.0}
{'IMI': 100.0, 'INJAS': 100.0, 'ISCAL': 100.0, 'STD_': 0.0, 'SR': 0.0}
{'RAO/RAE': 100.0, 'STACH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ISC_': 100.0, 'LVH': 100.0, 'SR': 0.0}
{'IRBBB': 100.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SARRH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 35.0, 'STD_': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IPLMI': 100.0, 'CRBBB': 100.0, 

{'IMI': 15.0, 'NST_': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'LMI': 100.0, 'LPR': 0.0, 'QWAVE': 0.0, 'LOWT': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'IMI': 50.0, 'LVH': 100.0, 'ISC_': 100.0, 'LAO/LAE': 100.0, 'STD_': 0.0, 'SBRAD': 0.0}
{'ISCLA': 100.0, 'LOWT': 0.0, 'SR': 0.0}
{'CRBBB': 100.0, '2AVB': 100.0}
{'LAFB': 100.0, 'NST_': 0.0, 'SR': 0.0}
{'ASMI': 100.0, 'IMI': 15.0, 'LAFB': 100.0, 'PAC': 0.0, 'QWAVE': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 50.0}
{'CLBBB': 100.0, '1AVB': 100.0, 'SBRAD': 0.0}
{'NORM': 100.0, 'AFIB': 0.0}
{'NST_': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'SEHYP': 100.0, 'QWAVE': 0.0, 'SR': 0.0}
{'IMI': 50.0, 'NDT': 100.0, 'LOWT': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'HVOLT': 0.0, 'SR': 0.0}
{'LAFB': 80.0, 'PAC': 0.0, 'SR': 0.0}
{'IMI': 35.0, 'NDT': 100.0, 'STACH': 0.0}
{'NDT': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'INJAS': 100.0, 'INJAL': 100.0, 'LVH': 100.0, 'STD_': 0.0, 'INVT

{'ASMI': 15.0, 'LVH': 50.0, 'ABQRS': 0.0, 'VCLVH': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SBRAD': 0.0}
{'IMI': 15.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NST_': 100.0, 'AFIB': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'WPW': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'LAFB': 100.0, '1AVB': 100.0, 'SR': 0.0}
{'WPW': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 50.0, 'PVC': 100.0, 'VCLVH': 0.0}
{'NORM': 50.0, 'LVOLT': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IRBBB': 80.0, 'SR': 0.0}
{'ASMI': 100.0, 'ISCLA': 100.0, 'IVCD': 100.0, 'LAO/LAE': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IMI': 100.0, 'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'SR': 0.0}
{'CRBBB': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'IVCD': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'IPMI': 100.0, 'LVH': 50.0, 'ISC_': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'ALMI': 50.0, '

{'NORM': 100.0, 'SR': 0.0}
{'ILMI': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'STACH': 0.0}
{'IMI': 100.0, 'ASMI': 100.0, 'PVC': 100.0, 'ABQRS': 0.0, 'AFIB': 0.0}
{'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 50.0, '1AVB': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, '1AVB': 100.0, 'SR': 0.0}
{'PACE': 100.0}
{'ASMI': 100.0, 'ALMI': 100.0, 'LAFB': 100.0, 'SR': 0.0}
{'LVH': 50.0, 'VCLVH': 0.0, 'SVARR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'LVH': 100.0, 'ISC_': 100.0, 'ILBBB': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'ISCAS': 100.0, 'ISCAL': 100.0, 'SR': 0.0}
{'IMI': 100.0, '1AVB': 100.0, 'PVC': 100.0, 'BIGU': 100.0, 'STACH': 0.0}
{'PACE': 100.0}
{'NORM': 100.0, 'ABQRS': 0.0, 'SVARR': 0.0}
{'ASMI': 100.0, 'ILMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'NST_': 100.0, 'SR': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'LMI': 15.0, 'CLBBB': 100.0, '1AVB': 100.0, 'SARRH': 0.0}
{

{'ISCIL': 100.0, 'PVC': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'ASMI': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IMI': 100.0, 'ASMI': 100.0, 'ABQRS': 0.0, 'SBRAD': 0.0}
{'ISCAL': 100.0, 'ISCIL': 100.0, 'AFIB': 0.0}
{'ISCIN': 50.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'SR': 0.0}
{'IRBBB': 100.0, 'LVH': 100.0, 'ISC_': 100.0, 'VCLVH': 0.0, 'AFIB': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NDT': 100.0, 'AFIB': 0.0}
{'NORM': 15.0, 'SR': 0.0}
{'NDT': 100.0, 'AFIB': 0.0}
{'NORM': 80.0, 'SR': 0.0}
{'ASMI': 50.0, 'ILBBB': 100.0, 'LAFB': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'IVCD': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NST_': 100.0, 'AFIB': 0.0}
{'NDT': 100.0, 'PVC': 100.0}
{'NORM': 50.0, 'STACH': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'NORM': 100.0, 'SR': 0.0}
{'CLBBB': 100.0, 'LAO/LAE': 100.0, 'SR': 0.

In [12]:
agg_df[['diagnostic','SCP-ECG Statement Description', 'AHA code', 'aECG REFID', 'CDISC Code',
       'DICOM Code']]

,diagnostic,SCP-ECG Statement Description,AHA code,aECG REFID,CDISC Code,DICOM Code
NDT,1.0,non-diagnostic T abnormalities,NaN,NaN,NaN,NaN
NST_,1.0,non-specific ST changes,145.0,MDC_ECG_RHY_STHILOST,NaN,NaN
DIG,1.0,suggests digitalis-effect,205.0,NaN,NaN,NaN
LNGQT,1.0,long QT-interval,148.0,NaN,NaN,NaN
NORM,1.0,normal ECG,1.0,NaN,NaN,F-000B7
IMI,1.0,inferior myocardial infarction,161.0,NaN,NaN,NaN
ASMI,1.0,anteroseptal myocardial infarction,165.0,NaN,NaN,NaN
LVH,1.0,left ventricular hypertrophy,142.0,NaN,C71076,NaN
LAFB,1.0,left anterior fascicular block,101.0,MDC_ECG_BEAT_BLK_ANT_L_HEMI,C62267,D3-33140
ISC_,1.0,ischemic ST-T changes,226.0,NaN,NaN,NaN


In [13]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21837 entries, 1 to 21837
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   patient_id                    21837 non-null  float64
 1   age                           21748 non-null  float64
 2   sex                           21837 non-null  int64  
 3   height                        6983 non-null   float64
 4   weight                        9429 non-null   float64
 5   nurse                         20328 non-null  float64
 6   site                          21819 non-null  float64
 7   device                        21837 non-null  object 
 8   recording_date                21837 non-null  object 
 9   report                        21837 non-null  object 
 10  scp_codes                     21837 non-null  object 
 11  heart_axis                    13332 non-null  object 
 12  infarction_stadium1           5626 non-null   object 
 13  i

In [14]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, NDT to 2AVB
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   description                    44 non-null     object 
 1   diagnostic                     44 non-null     float64
 2   form                           4 non-null      float64
 3   rhythm                         0 non-null      float64
 4   diagnostic_class               44 non-null     object 
 5   diagnostic_subclass            44 non-null     object 
 6   Statement Category             44 non-null     object 
 7   SCP-ECG Statement Description  44 non-null     object 
 8   AHA code                       22 non-null     float64
 9   aECG REFID                     12 non-null     object 
 10  CDISC Code                     13 non-null     object 
 11  DICOM Code                     9 non-null      object 
dtypes: float64(4), object(8)
memory usage: 5.7+ KB


In [15]:
agg_df['diagnostic_subclass'].unique()

array(['STTC', 'NST_', 'NORM', 'IMI', 'AMI', 'LVH', 'LAFB/LPFB', 'ISC_',
       'IRBBB', '_AVB', 'IVCD', 'ISCA', 'CRBBB', 'CLBBB', 'LAO/LAE',
       'ISCI', 'LMI', 'RVH', 'RAO/RAE', 'WPW', 'ILBBB', 'SEHYP', 'PMI'],
      dtype=object)

In [16]:
# 'AFIB': 100.0 - аритмия
# 'SBRAS' - брадикардия?
# по остальным вообще информации нет - лейблы (а в agg описание тоже не подходит, в основном про миокард и инфаркт и тд)


In [17]:
# наши таргеты содержатся в report и будут доступны после парсинга, разораться в них поможет суперкласс
Y.report

ecg_id
1                   sinusrhythmus periphere niederspannung
2                      sinusbradykardie sonst normales ekg
3                               sinusrhythmus normales ekg
4                               sinusrhythmus normales ekg
5                               sinusrhythmus normales ekg
                               ...                        
21833    ventrikulÄre extrasystole(n) sinustachykardie ...
21834    sinusrhythmus lagetyp normal qrs(t) abnorm    ...
21835    sinusrhythmus lagetyp normal t abnorm in anter...
21836    supraventrikulÄre extrasystole(n) sinusrhythmu...
21837    sinusrhythmus p-sinistrocardiale lagetyp norma...
Name: report, Length: 21837, dtype: object

In [18]:
# посмотрим, какие варианты у суперкласса
Y['diagnostic_superclass'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[pathology]          11902
[NORM]                9083
[pathology, NORM]      445
[]                     407
Name: diagnostic_superclass, dtype: int64

In [19]:
data_X = load_raw_data_X(Y, sampling_rate, path)

In [20]:
# уберем пустые значения суперкласса, а также те, которые будут нас путать (нормальные ЭКГ, смешанные с ненормальными)
# оставим два основных для нас информационных столбца: нормальное/не нормальное ЭКГ и патология/нет
norm = []
pathology = []
drop = []
for ind, i in enumerate(Y['diagnostic_superclass']): 
    if len(i)==0:
        drop.append(ind+1)
    elif len(i) > 1 and 'NORM' in i:
        drop.append(ind+1)     
    elif 'NORM' in i:
        norm.append(ind+1)
    elif 'pathology' in i:
        pathology.append(ind+1)
        
Y = Y.drop([d for d in drop])
Y['norm_ecg'] = 0
Y['pathology'] = 0

for ind in Y.index:
    if ind in norm:
        Y['norm_ecg'][ind] = 1 
    elif ind in pathology:
        Y['pathology'][ind] = 1    

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
Y['ecg_id'] = Y.index

In [22]:
Y.index = range(0, Y.shape[0])

In [23]:
Y[['diagnostic_superclass', 'norm_ecg', 'pathology', 'ecg_id']]

,diagnostic_superclass,norm_ecg,pathology,ecg_id
0,[NORM],1,0,1
1,[NORM],1,0,2
2,[NORM],1,0,3
3,[NORM],1,0,4
4,[NORM],1,0,5
...,...,...,...,...
20980,[pathology],0,1,21833
20981,[NORM],1,0,21834
20982,[pathology],0,1,21835
20983,[NORM],1,0,21836


In [24]:
# первая контрольная точка (патология/нормальное ЭКГ)
Y.to_csv('Y.csv')
Y = pd.read_csv('Y.csv')

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
Y.shape

(20985, 32)

In [26]:
# наш таргет - report - не забыть обработать
Y[Y['norm_ecg']==0].report

7        sinusrhythmus linkstyp qrs(t) abnormal    infe...
18       sinusrhythmus linkstyp unspezifisches abnormal...
21       sinusrhythmus verdacht auf p-sinistrocardiale ...
23       sinusrhythmus verdacht auf p-sinistrocardiale ...
25       sinusrhythmus unspezifischer intraventrikulÄre...
                               ...                        
20975    ventrikulÄre extrasystole(n), trigeminus supra...
20976    ventrikulÄre extrasystole(n) sinusrhythmus lin...
20979    sinusrhythmus linkstyp periphere niederspannun...
20980    ventrikulÄre extrasystole(n) sinustachykardie ...
20982    sinusrhythmus lagetyp normal t abnorm in anter...
Name: report, Length: 11902, dtype: object

In [27]:
# чтобы определить главные таргеты из report, вынесем все в признаки в отдельный файл
target_values = []
target_ill = []

for ind, target in enumerate(Y[Y['norm_ecg']==0].report):
    for key, value in target_dict.items():
             for v in value:           
                if v in target:
                    target_values.append([key, ind])
                    target_ill.append(target)

In [28]:
# проверить, что все возможные варианты report учтены
for ind, target in enumerate(Y[Y['norm_ecg']==0].report):
    if target not in target_ill:
        for key, value in target_dict.items():
             for v in value:           
                if v not in target:
                    print(target)

In [29]:
# наши целевые таргеты

sinus_rythm = []
arithmia = []
bradycardia = []
tachycardia = []
fibrillation = []
nan_ill = []
questions_ill = []

for target in target_values:
    if target[0] == 'sinus' and target[1] not in sinus_rythm:
        sinus_rythm.append(target[1])
    elif target[0] == 'arithmia' and target[1] not in arithmia:
        arithmia.append(target[1])
    elif target[0] == 'bradykardie' and target[1] not in bradycardia:
        bradycardia.append(target[1])
    elif target[0] == 'tahicardia' and target[1] not in tachycardia:
        tachycardia.append(target[1])
    elif target[0] == 'fibrillation' and target[1] not in fibrillation:
        fibrillation.append(target[1])
    elif target[0] == 'Nans' and target[1] not in nan_ill:
        nan_ill.append(target[1])
    elif target[0] == 'dont_now' and target[1] not in questions_ill:
        questions_ill.append(target[1])

Y['sinus_rythm'] = 0
Y['arithmia'] = 0
Y['bradycardia'] = 0
Y['tachycardia'] = 0
Y['fibrillation'] = 0
Y['nan_ill'] = 0
Y['questions_ill'] = 0

add_col = ['sinus_rythm', 'arithmia', 'bradycardia', 'tachycardia', 'fibrillation', 'nan_ill', 'questions_ill']
add_values = {'sinus_rythm': sinus_rythm, 
              'arithmia': arithmia, 
              'bradycardia' :bradycardia, 
              'tachycardia': tachycardia, 
              'fibrillation': fibrillation,
              'nan_ill': nan_ill, 
              'questions_ill': questions_ill}


In [30]:
for ind in sinus_rythm:
    Y['sinus_rythm'][ind] = 1
for ind in arithmia:
    Y['arithmia'][ind] = 1
for ind in bradycardia:
    Y['bradycardia'][ind] = 1
for ind in tachycardia:
    Y['tachycardia'][ind] = 1
for ind in fibrillation:
    Y['fibrillation'][ind] = 1
for ind in questions_ill:
    Y['questions_ill'][ind] = 1
        

/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/natalaurina/.pyenv/versions/3.7.8/envs/jupyter_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

In [31]:
Y[['sinus_rythm', 'arithmia', 'bradycardia', 'tachycardia', 'fibrillation', 'nan_ill', 'questions_ill',
   'norm_ecg', 'report', 'ecg_id']][Y['norm_ecg']==0].groupby('fibrillation').head(2000)

,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation,nan_ill,questions_ill,norm_ecg,report,ecg_id
7,1,0,0,0,0,0,0,0,sinusrhythmus linkstyp qrs(t) abnormal infe...,8
18,1,0,0,0,0,0,0,0,sinusrhythmus linkstyp unspezifisches abnormal...,22
21,1,0,0,0,0,0,0,0,sinusrhythmus verdacht auf p-sinistrocardiale ...,26
23,1,0,0,0,0,0,0,0,sinusrhythmus verdacht auf p-sinistrocardiale ...,28
25,1,0,0,0,0,0,0,0,sinusrhythmus unspezifischer intraventrikulÄre...,30
...,...,...,...,...,...,...,...,...,...,...
11856,0,0,0,1,1,0,1,0,sinus rhythm. possible left atrial enlargement...,12318
11870,0,0,0,1,1,0,1,0,premature ventricular contraction(s). sinus rh...,12333
11875,0,0,0,0,1,0,1,0,sinusbradykardie llt kammerhypertrophie lvh si...,12338
11880,0,0,0,0,1,0,1,0,atrial fibrillation. st segments are depressed...,12343


In [46]:
ill_df = Y[['sinus_rythm', 'arithmia', 'bradycardia', 'tachycardia', 'fibrillation', 'ecg_id']][Y['norm_ecg']==0]

In [48]:
ill_df.to_csv('ill_data.csv', index=False)
ill_data = pd.read_csv('ill_data.csv')
ill_data

,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation,ecg_id
0,1,0,0,0,0,8
1,1,0,0,0,0,22
2,1,0,0,0,0,26
3,1,0,0,0,0,28
4,1,0,0,0,0,30
...,...,...,...,...,...,...
11897,0,0,0,0,0,21827
11898,0,0,0,0,0,21828
11899,0,0,0,0,0,21832
11900,0,0,0,0,0,21833


In [50]:
Y.to_csv('Y_current.csv', index=False)


In [73]:
# надо будет удалить экг, где неясно, к чему относится таргет
drop_data = Y[['ecg_id', 'questions_ill']][(Y['norm_ecg']==0)&(Y['questions_ill']==1)]
drop_ecg_id = drop_data['ecg_id'].tolist()
len(drop_ecg_id)

3295

In [34]:
train_signal = pd.read_csv('train_signal.csv')
test_signal = pd.read_csv('test_signal.csv')
valid_signal = pd.read_csv('valid_signal.csv')


In [35]:
# к каждому фрейму присоединить таргеты
train_df = train_signal.join(ill_df.set_index('ecg_id'), on='ecg_id')
test_df = test_signal.join(ill_df.set_index('ecg_id'), on='ecg_id')
valid_df = valid_signal.join(ill_df.set_index('ecg_id'), on='ecg_id')
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)
valid_df = valid_df.fillna(0)

In [70]:
# надо удалить экг, где неясно, к чему относится таргет
train_df.drop(pd.Index(np.where(train_df['ecg_id'].isin([i for i in drop_ecg_id]))[0]), inplace = True)
test_df.drop(pd.Index(np.where(test_df['ecg_id'].isin([i for i in drop_ecg_id]))[0]), inplace = True)
valid_df.drop(pd.Index(np.where(valid_df['ecg_id'].isin([i for i in drop_ecg_id]))[0]), inplace = True)


In [69]:
train_df.head()

,ecg_id,channel-0,channel-1,channel-2,channel-3,channel-4,channel-5,channel-6,channel-7,channel-8,channel-9,channel-10,channel-11,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation
0,1,-0.119,-0.055,0.064,0.086,-0.091,0.004,-0.069,-0.031,0.000,-0.026,-0.039,-0.079,0.0,0.0,0.0,0.0,0.0
1,1,-0.116,-0.051,0.065,0.083,-0.090,0.006,-0.064,-0.036,-0.003,-0.031,-0.034,-0.074,0.0,0.0,0.0,0.0,0.0
2,1,-0.120,-0.044,0.076,0.082,-0.098,0.016,-0.058,-0.034,-0.010,-0.028,-0.029,-0.069,0.0,0.0,0.0,0.0,0.0
3,1,-0.117,-0.038,0.080,0.077,-0.098,0.021,-0.050,-0.030,-0.015,-0.023,-0.022,-0.064,0.0,0.0,0.0,0.0,0.0
4,1,-0.103,-0.031,0.072,0.066,-0.087,0.021,-0.045,-0.027,-0.020,-0.019,-0.018,-0.058,0.0,0.0,0.0,0.0,0.0


In [71]:
# сохранить локально готовые данные теста, валидации и тренировочные
train_df.to_csv('train_df.csv', index=False)
train_df = pd.read_csv('train_df.csv')
train_df

,ecg_id,channel-0,channel-1,channel-2,channel-3,channel-4,channel-5,channel-6,channel-7,channel-8,channel-9,channel-10,channel-11,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation
0,1,-0.119,-0.055,0.064,0.086,-0.091,0.004,-0.069,-0.031,0.000,-0.026,-0.039,-0.079,0.0,0.0,0.0,0.0,0.0
1,1,-0.116,-0.051,0.065,0.083,-0.090,0.006,-0.064,-0.036,-0.003,-0.031,-0.034,-0.074,0.0,0.0,0.0,0.0,0.0
2,1,-0.120,-0.044,0.076,0.082,-0.098,0.016,-0.058,-0.034,-0.010,-0.028,-0.029,-0.069,0.0,0.0,0.0,0.0,0.0
3,1,-0.117,-0.038,0.080,0.077,-0.098,0.021,-0.050,-0.030,-0.015,-0.023,-0.022,-0.064,0.0,0.0,0.0,0.0,0.0
4,1,-0.103,-0.031,0.072,0.066,-0.087,0.021,-0.045,-0.027,-0.020,-0.019,-0.018,-0.058,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14864995,21836,0.096,0.189,0.093,-0.143,0.001,0.141,-0.034,0.189,0.162,0.227,0.183,0.128,0.0,0.0,0.0,0.0,0.0
14864996,21836,0.051,0.119,0.068,-0.085,-0.009,0.093,-0.049,0.195,0.172,0.217,0.127,0.081,0.0,0.0,0.0,0.0,0.0
14864997,21836,0.033,0.070,0.037,-0.052,-0.002,0.054,-0.097,0.140,0.215,0.218,0.101,0.052,0.0,0.0,0.0,0.0,0.0
14864998,21836,0.027,0.082,0.055,-0.055,-0.014,0.068,-0.104,0.123,0.200,0.210,0.335,0.010,0.0,0.0,0.0,0.0,0.0


In [72]:
test_df.to_csv('test_df.csv', index=False)
valid_df.to_csv('valid_df.csv', index=False)


In [40]:
test_df

,ecg_id,channel-0,channel-1,channel-2,channel-3,channel-4,channel-5,channel-6,channel-7,channel-8,channel-9,channel-10,channel-11,sinus_rythm,arithmia,bradycardia,tachycardia,fibrillation
0,9,-0.145,-0.097,0.049,0.121,-0.096,-0.024,-0.579,-0.818,-0.647,-0.452,-0.269,-0.032,0.0,0.0,0.0,0.0,0.0
1,9,-0.115,-0.083,0.032,0.099,-0.073,-0.025,-0.369,-0.523,-0.454,-0.339,-0.246,-0.041,0.0,0.0,0.0,0.0,0.0
2,9,-0.064,-0.060,0.005,0.062,-0.034,-0.027,-0.088,-0.147,-0.196,-0.188,-0.199,-0.040,0.0,0.0,0.0,0.0,0.0
3,9,-0.093,-0.101,-0.008,0.097,-0.042,-0.054,-0.006,-0.052,-0.136,-0.146,-0.164,-0.037,0.0,0.0,0.0,0.0,0.0
4,9,-0.081,-0.108,-0.027,0.095,-0.027,-0.067,0.028,-0.001,-0.098,-0.097,-0.148,-0.034,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2202995,21826,-0.056,-0.012,0.044,0.034,-0.050,0.016,0.064,-0.045,-0.065,-0.038,-0.014,-0.017,0.0,0.0,0.0,0.0,0.0
2202996,21826,-0.058,-0.057,0.001,0.057,-0.029,-0.028,0.072,-0.066,-0.071,-0.063,-0.057,-0.061,0.0,0.0,0.0,0.0,0.0
2202997,21826,-0.109,-0.054,0.055,0.081,-0.081,0.000,0.101,-0.049,-0.069,-0.057,-0.082,-0.090,0.0,0.0,0.0,0.0,0.0
2202998,21826,-0.091,-0.067,0.024,0.079,-0.057,-0.022,0.170,0.037,0.014,0.003,-0.047,-0.085,0.0,0.0,0.0,0.0,0.0
